In [1]:
!curl -O -L "https://cli-server-trusted-artifact-signer.apps.rosa.mmortari-rosa2.7bcr.p3.openshiftapps.com/clients/linux/cosign-amd64.gz"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 24.7M  100 24.7M    0     0   287M      0 --:--:-- --:--:-- --:--:--  287M


In [2]:
# using the RH DOWNSTREAM version
!gunzip cosign-amd64.gz
!mv cosign-amd64 /opt/app-root/bin/cosign
!chmod +x /opt/app-root/bin/cosign

In [3]:
!cosign version

  ______   ______        _______. __    _______ .__   __.
 /      | /  __  \      /       ||  |  /  _____||  \ |  |
|  ,----'|  |  |  |    |   (----`|  | |  |  __  |   \|  |
|  |     |  |  |  |     \   \    |  | |  | |_ | |  . `  |
|  `----.|  `--'  | .----)   |   |  | |  |__| | |  |\   |
 \______| \______/  |_______/    |__|  \______| |__| \__|
cosign: A tool for Container Signing, Verification and Storage in an OCI registry.

GitVersion:    5c43d256
GitCommit:     5c43d256f1391f505ada9f9c46c3e31b6e219c8e
GitTreeState:  clean
BuildDate:     2025-11-19T10:04:54Z
GoVersion:     go1.24.6 (Red Hat 1.24.6-1.el9_6) X:strictfipsruntime
Compiler:      gc
Platform:      linux/amd64



In [4]:
!cosign initialize --mirror=$TUF_URL --root=$TUF_URL/root.json

Root status: 
 {
	"local": "/opt/app-root/src/.sigstore/root",
	"remote": "https://tuf-trusted-artifact-signer.apps.rosa.mmortari-rosa2.7bcr.p3.openshiftapps.com",
	"metadata": {
		"root.json": {
			"version": 1,
			"len": 4128,
			"expiration": "12 Nov 26 12:19 UTC",
			"error": ""
		},
		"snapshot.json": {
			"version": 1,
			"len": 994,
			"expiration": "12 Nov 26 12:19 UTC",
			"error": ""
		},
		"targets.json": {
			"version": 1,
			"len": 2416,
			"expiration": "12 Nov 26 12:19 UTC",
			"error": ""
		},
		"timestamp.json": {
			"version": 1,
			"len": 995,
			"expiration": "12 Nov 26 12:19 UTC",
			"error": ""
		}
	},
	"targets": [
		"rekor.pub",
		"fulcio_v1.crt.pem",
		"trusted_root.json",
		"tsa.certchain.pem",
		"ctfe.pub"
	]
}


In [5]:
import base64
import json
import subprocess
import sys
import os

def decode_jwt_payload(token):
    """Decode JWT payload from base64url encoding."""
    parts = token.split('.')
    if len(parts) != 3:
        raise ValueError("Invalid JWT token format")
    
    payload_base64url = parts[1]
    padding = 4 - (len(payload_base64url) % 4)
    if padding != 4:
        payload_base64url += '=' * padding
    
    payload_base64 = payload_base64url.replace('-', '+').replace('_', '/')
    payload_bytes = base64.b64decode(payload_base64)
    
    return json.loads(payload_bytes)

# we assume standard location:
token_path = '/var/run/secrets/kubernetes.io/serviceaccount/token'

try:
    with open(token_path, 'r') as f:
        token = f.read().strip()
except FileNotFoundError:
    print(f"Error: Token file not found at {token_path}")
    # Simply stop here - subsequent cells can still run
    raise

payload = decode_jwt_payload(token)
oidc_issuer = payload.get('iss')
if not oidc_issuer:
    print("Error: 'iss' claim not found in JWT")
    raise ValueError("Missing 'iss' claim")

print(f"OIDC Issuer: {oidc_issuer}")

OIDC Issuer: https://rh-oidc.s3.us-east-1.amazonaws.com/23c734st3pn7l167mq97d0ot8848lgrl


Note that we need to manage the OCI registry credentials from the ODH Connection:

In [6]:
import tempfile
from contextlib import contextmanager, suppress
from collections.abc import Generator
from typing import TextIO
from pathlib import Path

@contextmanager
def temp_auth_file(auth: str | None) -> Generator[TextIO | None, None, None]:
    """Create a temporary auth file with optional auth data.

    If auth is None, yields None. Otherwise creates a temporary JSON file
    containing the auth string and yields the file handle.
    """
    if auth is None:
        yield None
    else:
        # Yield the file after closing so it can be read by another process,
        # and use delete=False so closing doesn't delete the file. We add our
        # own cleanup code to achieve an effect similar to using
        # delete=True, delete_on_close=False with Python 3.12 or later.
        config_path: str | None = None
        try:
            with tempfile.TemporaryDirectory(delete=False) as tmpdir:
                config_path = Path(tmpdir) / "config.json"
                config_path.write_text(auth)
            yield config_path  # type: ignore[misc]
        finally:
            if config_path is not None:
                with suppress(OSError):
                    os.unlink(config_path)

In [7]:
image = 'quay.io/mmortari/demo20251023:latest'

cmd = [
    'cosign', 'sign', '-y',
    f'--fulcio-url={os.environ["FULCIO_URL"]}',
    f'--rekor-url={os.environ["REKOR_URL"]}',
    f'--identity-token={token}',
    image
]

with temp_auth_file(os.environ[".dockerconfigjson"]) as auth_file:
    env = os.environ.copy()
    env['DOCKER_CONFIG'] = str(auth_file.parent)  # note this is transitive of cosign (https://github.com/search?q=repo%3Agoogle%2Fgo-containerregistry%20DOCKER_CONFIG&type=code)
    
    try:
        result = subprocess.run(
            cmd,
            env=env,
            check=True,
            capture_output=True,
            text=True
        )
        if result.stdout:
            print("Output:", result.stdout)
        if result.stderr:
            print("Stderr:", result.stderr)
        
    except subprocess.CalledProcessError as e:
        print(f"exit code: {e.returncode}")
        print(f"Stdout: {e.stdout}")
        print(f"Stderr: {e.stderr}")
        raise

Stderr: Generating ephemeral keys...
Retrieving signed certificate...
Successfully verified SCT...
    This can lead you to sign a different image than the intended one. Please use a
    digest (example.com/ubuntu@sha256:abc123...) rather than tag
    (example.com/ubuntu:latest) for the input to cosign. The ability to refer to
    images by tag will be removed in a future release.


	The sigstore service, hosted by sigstore a Series of LF Projects, LLC, is provided pursuant to the Hosted Project Tools Terms of Use, available at https://lfprojects.org/policies/hosted-project-tools-terms-of-use/.
	Note that if your submission includes personal data associated with this signed artifact, it will be part of an immutable record.
	This may include the email address associated with the account with which you authenticate your contractual Agreement.
	This information will be used for signing this artifact and will be stored in public transparency logs and cannot be removed later, and is subject

In [9]:
try:
    result = subprocess.run(
        ['cosign', 'tree', image],
        check=True,
        capture_output=True,
        text=True
    )
    if result.stdout:
        print("Output:", result.stdout)
    if result.stderr:
        print("Stderr:", result.stderr)
    
except subprocess.CalledProcessError as e:
    print(f"exit code: {e.returncode}")
    print(f"Stdout: {e.stdout}")
    print(f"Stderr: {e.stderr}")
    raise

Output: 📦 Supply Chain Security Related artifacts for an image: quay.io/mmortari/demo20251023:latest
└── 🔐 Signatures for an image tag: quay.io/mmortari/demo20251023:sha256-5def9dcc30b29d2ddd646b5a998cd7426cec958e990f810acfed7d64c80f149e.sig
   ├── 🍒 sha256:b032c8517d487aab01916466bf7e6b44a070de16cf8bba336a2a4a9cc58efa28
   ├── 🍒 sha256:b032c8517d487aab01916466bf7e6b44a070de16cf8bba336a2a4a9cc58efa28
   ├── 🍒 sha256:b032c8517d487aab01916466bf7e6b44a070de16cf8bba336a2a4a9cc58efa28
   └── 🍒 sha256:b032c8517d487aab01916466bf7e6b44a070de16cf8bba336a2a4a9cc58efa28

